In [ ]:
from bs4 import BeautifulSoup
import requests

res = {'title': [], 'date': [], 'url': []}

for i in range(1, 16):
    html = requests.get(f"https://sw7up.cbnu.ac.kr/community/notice?page={i}&limit=10&sort=-createdAt").text
    bsObject = BeautifulSoup(html, "html.parser")

    for link in bsObject.find_all('span', 'mb-0'):
        print(str(link).split('>')[1].split('<')[0])
        res['title'].append(str(link).split('>')[1].split('<')[0])

In [ ]:
from bs4 import BeautifulSoup
import requests

for i in range(1, 16):
    html = requests.get(f"https://sw7up.cbnu.ac.kr/community/notice?page={i}&limit=10&sort=-createdAt").text
    bsObject = BeautifulSoup(html, "html.parser")

    for link in bsObject.find_all('small')[2::3]:
        print(str(link).split('>')[1].split('.<')[0])
        res['date'].append(str(link).split('>')[1].split('.<')[0])

In [ ]:
%pip install selenium
%pip install webdriver_manager
%pip install firebase_admin

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
browser = webdriver.Chrome(ChromeDriverManager().install())

for i in range(1,16):                                                                          
    browser.get(f"https://sw7up.cbnu.ac.kr/community/notice?page={i}&limit=10&sort=-createdAt")
    time.sleep(5)

    for i in range(10):
        elem = browser.find_elements(By.CLASS_NAME, 'card')
        elem[i].click()
        time.sleep(5)
        
        res['url'].append(browser.current_url)
        browser.back()
        time.sleep(5)

In [5]:
import firebase_admin
from firebase_admin import credentials

cred = credentials.Certificate("<your private key>")
firebase_admin.initialize_app(cred)


In [8]:
res['title'] = res['title'][::-1]
res['url'] = res['url'][::-1]
res['date'] = res['date'][::-1]

res['title'] = res['title'][2:]
res['url'] = res['url'][2:]
res['date'] = res['date'][2:]

In [11]:
import pandas as pd

data = pd.DataFrame(res)
print(data)
data.to_csv('C:\\Users\\SAMSUNG\\Desktop\\crawl\\data.csv')         #크롤링 데이터를 csv 파일에 넣는다

                                                 title      date  \
0                                2022년 1학기 학습동아리 선발 안내  22.03.01   
1    2022년 K-SW스퀘어 여름방학(8주) / 가을학기(16주) 프로그램 서류심사 합...  22.03.02   
2                         2022년 창업 액셀러레이션 참가팀 모집(~4/1)  22.03.02   
3     2022년 SW중심대학사업단 Git-Github 기초/고급 특강 신청 안내(~3/11)  22.03.04   
4                            2022년 1학기 학부생 TA 추가 모집 안내  22.03.04   
..                                                 ...       ...   
143               2022학년도 2학기 산학활동참여 마일리지 장학생 선발 결과 안내  22.11.21   
144        2022년 SW중심대학사업단 글로벌캡스톤디자인(CEDC 2022) 합격자 안내  22.11.28   
145  예비대학생을 위한 2023학년도 입학 전 SW기초교육 '컴퓨팅 사고력 기르기' 신청 안내  22.12.08   
146                       2023년 OSS운영보조원 모집 안내(~12/20)  22.12.08   
147                        2022년 클라우드 특강 신청 안내(~12/15)  22.12.08   

                                                   url  
0    https://sw7up.cbnu.ac.kr/community/notice/621d...  
1    https://sw7up.cbnu.ac.kr/community/notice/621e...  
2    https:/

In [ ]:
import pandas as pd
res = pd.read_csv('./data.csv')
print(res)

In [13]:
from firebase_admin import firestore

db = firestore.client()
for i in range(148):
    doc_ref = db.collection(u'sw7up').document(u'%04d' % i)
    doc_ref.set({'title': res['title'][i], 
                'date': res['date'][i],
                'url': res['url'][i]})